In [5]:
import math
import numpy

def periodic(n):
    global down
    if (n % 2 == 0): 
        return float(0.0)
    elif (down == 1): 
        down = 0
        return float(1.0)     
    elif (down == 0):
        down = 1
        return float(-1.0)

def activation_tanh(x):
    return math.tanh(x)

def derivative_activation_tanh(x):
    return 1.0 / pow(math.cosh(x), 2)

def forward():
    global W, v_input, WCHH, v_context_hidden, WCOH, context_output, T, v_hidden, W_, T_, output
    for j in range(number_of_neurons):
        S = 0.0
        for i in range(number_of_rows):
            S += W[i][j] * v_input[i]
        for i in range(number_of_neurons):
            S += WCHH[i][j] * v_context_hidden[i]
        S += WCOH[0][j] * context_output
        S -= T[j]
        v_hidden[j] = activation_tanh(S)
    S = 0.0
    for i in range(number_of_neurons):
        S += W_[i][0] * v_hidden[i]
    S -= T_
    output = activation_tanh(S)
    for i in range(number_of_neurons):
        v_context_hidden[i] = v_hidden[i]
    context_output = output
    
def back(val):
    global alpha, output, W, W_, v_hidden, v_input, WCHH, v_context_hidden, WCOH, context_output, T, T_
    for i in range(number_of_neurons):
        for j in range(number_of_rows):
            W[j][i] -= float(alpha * (output - val)) * W_[i][0] * derivative_activation_tanh(v_hidden[i]) * v_input[j]
        for j in range(number_of_neurons):
            WCHH[j][i] -= float(alpha * (output - val)) * W_[i][0] * derivative_activation_tanh(v_hidden[i]) * v_context_hidden[j]
        W_[i][0] -= float(alpha * (output - val)) * v_hidden[i]
        WCOH[0][i] -= float(alpha * (output - val)) * W_[i][0] * derivative_activation_tanh(v_hidden[i]) * context_output
        T[i] = float(alpha * (output - val)) * W_[i][0] * derivative_activation_tanh(v_hidden[i])
    T_ = float(alpha * (output - val))

print("Choose a prediction Sequence:")
choise = int(input("1. Sinus\n2. Cosinus\n3. Period\n\nChoose: "))
sequence_size = int(input("Sequence dimension: "))
number_of_elements = int(input("Number of sequence elements to predict: "))
down = 1
sequence = []
expected_sequence = []
if (choise == 1):
    for n in range(1, sequence_size + 1):
        sequence.append(math.sin(n))
    sequence = numpy.array(sequence, dtype="float64").tolist()
    for n in range(1, sequence_size + number_of_elements + 1):
        expected_sequence.append(math.sin(n))
    expected_sequence = numpy.array(expected_sequence, dtype="float64").tolist()
elif (choise == 2):
    for n in range(1, sequence_size + 1):
        sequence.append(float(math.cos(n)))
    sequence = numpy.array(sequence, dtype="float64").tolist()
    for n in range(1, sequence_size + number_of_elements + 1):
        expected_sequence.append(float(math.cos(n)))
    expected_sequence = numpy.array(expected_sequence, dtype="float64").tolist()
elif (choise == 3):
    for n in range(1, sequence_size + 1):
        sequence.append(periodic(n))
    sequence = numpy.array(sequence, dtype="float64").tolist()
    down = 1
    for n in range(1, sequence_size + number_of_elements + 1):
        expected_sequence.append(periodic(n))
    expected_sequence = numpy.array(expected_sequence, dtype="float64").tolist()

print("Source sequence: ", sequence)
print("Expected predicted sequence: ", expected_sequence)
number_of_rows = int(input("Number of rows in the learning matrix: "))
number_of_strings_in_matrix = sequence_size - number_of_rows
print("Number of strings in the learning matrix: ", number_of_strings_in_matrix)
number_of_neurons = int(input("Number of neurons in the hidden layer: "))
alpha = float(input("Learning step: "))
max_error = float(input("Maximum allowable error: "))
N = int(input("Maximum number of training steps: "))

v_input = numpy.zeros(number_of_rows).tolist()
v_hidden = numpy.zeros(number_of_neurons).tolist()
v_output = 0.0
v_context_hidden = numpy.zeros(number_of_neurons).tolist()
context_output = 0.0
T = numpy.zeros(number_of_neurons).tolist()
T_ = 0.0
X = numpy.zeros((number_of_strings_in_matrix, number_of_rows)).tolist()
expected_values = []

for i in range(number_of_strings_in_matrix):
    for j in range(number_of_rows):
        X[i][j] = sequence[i + j]
    expected_values.append(sequence[i + number_of_rows])

expected_values = numpy.array(expected_values, dtype="float64").tolist()
W = (numpy.random.rand(number_of_rows, number_of_neurons) * 2.0 - 1.0).tolist()
WCHH = (numpy.random.rand(number_of_neurons, number_of_neurons) * 2.0 - 1.0).tolist()
W_ = (numpy.random.rand(number_of_neurons, 1) * 2.0 - 1.0).tolist()
WCOH = (numpy.random.rand(1, number_of_neurons) * 2.0 - 1.0).tolist()

E = 100.0
iteration = 0
while E > max_error and iteration <= N:
    iteration += 1
    E = 0.0
    for i in range(number_of_strings_in_matrix):
        v_input = X[i]
        forward()
        E += (float(output - expected_values[i]) ** 2.0) / 2.0
        back(expected_values[i])
    if iteration % 100 == 0:
        print("Iterations: ", iteration, " |  Error: ", E)

result = []
j = sequence_size - number_of_rows
for i in range(number_of_rows):
    v_input[i] = sequence[j]
    j += 1
for i in range(number_of_elements):
    if i > 0:
        for j in range(number_of_rows - 1):
            v_input[j] = v_input[j + 1]
        v_input[number_of_rows - 1] = output
    forward()
    result.append(output)
print("Result after testing:")
for i in range(number_of_elements):
    print("Expected value: ", expected_sequence[sequence_size + i], "| Received value: ", result[i], "Error: ", expected_sequence[sequence_size + i] - result[i])

Choose a prediction Sequence:
Source sequence:  [0.8414709848078965, 0.9092974268256817, 0.1411200080598672, -0.7568024953079282, -0.9589242746631385, -0.27941549819892586, 0.6569865987187891, 0.9893582466233818, 0.4121184852417566, -0.5440211108893698]
Expected predicted sequence:  [0.8414709848078965, 0.9092974268256817, 0.1411200080598672, -0.7568024953079282, -0.9589242746631385, -0.27941549819892586, 0.6569865987187891, 0.9893582466233818, 0.4121184852417566, -0.5440211108893698, -0.9999902065507035, -0.5365729180004349, 0.4201670368266409]
Number of strings in the learning matrix:  7
Iterations:  100  |  Error:  0.823760581941293
Iterations:  200  |  Error:  0.2774948270492599
Iterations:  300  |  Error:  0.259416435806001
Iterations:  400  |  Error:  0.22485914210701752
Iterations:  500  |  Error:  0.11694787236510644
Iterations:  600  |  Error:  0.0736519177282704
Iterations:  700  |  Error:  0.05431817275109403
Iterations:  800  |  Error:  0.04534366043902835
Iterations:  900 